In [ ]:
!pip install ts2vg

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# from ts2vg import *
from nltk import ngrams

In [18]:
shop_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/release_10_23_2020.csv')
shop_df.head()

,session_id_hash,event_type,product_action,product_skus_hash,server_timestamp_epoch_ms,hashed_url
0,00000005c19449b57d8d05dfc8b5474be0654032,pageview,NaN,NaN,1544469996111,da99729886aff70a02733b6cd69ee7df35622d9302347e...
1,00006a0ada94a5186163a25e9ed9c94481c820d9,pageview,NaN,NaN,1545737959865,e2f7e0cee4272e804f0d323a3513dd01716a5a40ab9abf...
2,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,NaN,NaN,1544908835402,ea7b2493be61ff454f8cce412f9dc281e605daec8c43b5...
3,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,NaN,NaN,1544908847263,ea7b2493be61ff454f8cce412f9dc281e605daec8c43b5...
4,00007d15aeb741b3cdd873cb3933351d699cc320,pageview,detail,90add4dd072e160034e98ddea257e0b59441eae00d8955...,1544909035879,8fa1ecf31ececb27ebe9c529966f3d1f907542fe138d5d...


In [19]:
print(shop_df.event_type.value_counts())
print(shop_df.product_action.value_counts())

event_type
pageview    4565253
event        868358
Name: count, dtype: int64
product_action
detail      1640190
add          743363
click         69831
remove        51512
purchase       9926
Name: count, dtype: int64


## Pre-processing

In [20]:
print(shop_df['product_action'].unique())
shop_df.drop(columns=['event_type', 'product_skus_hash', 'hashed_url'], axis=1, inplace=True)
shop_df.head()

[nan 'detail' 'click' 'add' 'purchase' 'remove']


,session_id_hash,product_action,server_timestamp_epoch_ms
0,00000005c19449b57d8d05dfc8b5474be0654032,NaN,1544469996111
1,00006a0ada94a5186163a25e9ed9c94481c820d9,NaN,1545737959865
2,00007d15aeb741b3cdd873cb3933351d699cc320,NaN,1544908835402
3,00007d15aeb741b3cdd873cb3933351d699cc320,NaN,1544908847263
4,00007d15aeb741b3cdd873cb3933351d699cc320,detail,1544909035879


1. Symbolisation

In [21]:
shop_df['product_action'].replace(np.nan,1, inplace=True)
symbols = {
    'detail': 2,
    'add': 3,
    'remove': 4,
    'purchase': 5,
    'click': 6
}
shop_df['product_action'].replace(symbols, inplace=True)

In [22]:
shop_df = shop_df.groupby(shop_df.session_id_hash).agg(list).reset_index()
print(shop_df.shape)
shop_df.head()

(443660, 3)


,session_id_hash,product_action,server_timestamp_epoch_ms
0,00000005c19449b57d8d05dfc8b5474be0654032,[1],[1544469996111]
1,00006a0ada94a5186163a25e9ed9c94481c820d9,[1],[1545737959865]
2,00007d15aeb741b3cdd873cb3933351d699cc320,"[1, 1, 2, 1, 2, 1, 2]","[1544908835402, 1544908847263, 1544909035879, ..."
3,0000c49d1359c87f112285d63a9fc0d4c50f73b4,"[1, 1, 1]","[1544780629084, 1544780652666, 1544780882467]"
4,0001135282dbaf0e734345212ea6f8da9f5b66ed,[1],[1544422592886]


2. Assign classes

In [23]:
shop_df['conversion_class'] = shop_df['product_action'].apply(lambda x: 'C' if 5 in x else 'NC')

3. Trimming such that L == [8,9,10,11,12,13,14,15,16,17]

In [24]:
# # Truncating till 8
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x[0:x.index(5)] if 5 in x else x)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) >= 6 else np.nan)
shop_df.dropna(inplace=True)
shop_df['product_action'] = shop_df['product_action'].apply(lambda x: x if len(x) <= 6 else np.nan)
shop_df.dropna(inplace=True)
print(shop_df.shape)
print(shop_df.conversion_class.value_counts())
shop_df.head()  

(16576, 4)
conversion_class
NC    16337
C       239
Name: count, dtype: int64


,session_id_hash,product_action,server_timestamp_epoch_ms,conversion_class
16,0003704e248795ae33f6cbe62b082cce4a011d4d,"[1, 1, 1, 1, 1, 1]","[1544260763106, 1544260787306, 1544260800780, ...",NC
31,0006312fb6c4940082a2dd04bd1ec06e3c26b7d9,"[1, 1, 2, 1, 2, 1]","[1545725372988, 1545725388695, 1545725506240, ...",NC
43,0007f4d284e653df33ecc6f21b4bd14f47afebf7,"[1, 1, 1, 2, 1, 2]","[1545133947443, 1545133999820, 1545134010449, ...",NC
44,0008452a3d02e0ceaeae74da1bcc5db67507aec7,"[1, 1, 2, 1, 2, 1]","[1544358279425, 1544358307043, 1544358389289, ...",NC
61,000ae863e71b64c88d1a24e84f511fa55b262a30,"[2, 2, 1, 1, 2, 1]","[1545680866514, 1545680889832, 1545680900307, ...",NC


## Feature Importance (HVGMs)

In [ ]:
def reduced_time(time):
    index = 0
    for sec in time:
        if index<len(time)-1:                           # do -2 when using all events
            time[index] = abs(time[index+1] - time[index])
        index+=1
    time.pop()

    # final_val = time.pop()
    # final_val = abs(final_val - avg)
    # time.append(final_val)
    
    total_time = sum(time)
    index = 0
    for sec in time:
        time[index] = sec/total_time
        index+=1
    
    return time

shop_preprocessed_df = shop_df.copy()    
shop_preprocessed_df['reduced_time'] = shop_preprocessed_df['server_timestamp_epoch_ms'].apply(lambda x: reduced_time(x))

In [ ]:
shop_preprocessed_df.reset_index(inplace=True)
shop_preprocessed_df.drop(['server_timestamp_epoch_ms', 'index'], axis=1, inplace=True)
shop_preprocessed_df.head()

In [ ]:
def generate_edges(time_list):
    vg = HorizontalVG()
    vg.build(time_list)
    edges = vg.edges
    return edges

shop_preprocessed_df['HVGms_edges'] = shop_preprocessed_df['reduced_time'].apply(lambda x: generate_edges(x))
shop_preprocessed_df.head()

In [ ]:
def pattern_hvg_4(edges):  
    i = 0
    edges.sort(key = lambda x: x[1])
    j = edges[len(edges)-1][1]
    pattern = list()
    while i<=int(j/2):
        edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+3]
        edges_tb_used = sorted(edges_tb_used, key=lambda x: (x[0], x[1]))
        # print(i)
        if len(edges_tb_used) == 5:
            if [(i, i+1), (i, i+3), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                pattern.append('F4')
            if [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('E4')
        
        elif len(edges_tb_used) == 4:
            if [(i, i+1), (i+1, i+2), (i+1, i+3), (i+2, i+3)] == edges_tb_used:
                pattern.append('D4')
            if [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('C4')
            if [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3)] == edges_tb_used:
                pattern.append('B4')
            
        elif len(edges_tb_used) == 3:
            pattern.append('A4')
        
        i+=1
                
    return pattern

In [ ]:
def pattern_hvg_5(edges):
    i = 0
    edges.sort(key = lambda x: x[1])
    j = edges[len(edges)-1][1]
    pattern = list()
    # print("=====")
    # print(edges)
    while i<=int(j/2):
        edges_tb_used = [edges for edges in edges if edges[0] >= i and edges[1] <= i+4]
        edges_tb_used = sorted(edges_tb_used, key=lambda x: (x[0], x[1]))

        if len(edges_tb_used) == 7:
            if [(i, i+1), (i, i+2), (i, i+4), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('V5')
            elif [(i, i+1), (i, i+3), (i, i+4), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('U5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+3), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('T5')
            elif [(i, i+1), (i, i+2), (i, i+3), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('S5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('R5')
                
        elif len(edges_tb_used) == 6:
            if [(i, i+1), (i+1, i+2), (i+1, i+3), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('Q5')
            elif [(i, i+1), (i, i+3), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('P5')
            elif [(i, i+1), (i, i+2), (i, i+3), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('O5')
            elif [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('N5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('M5')
            elif [(i, i+1), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('L5')
            elif [(i, i+1), (i, i+2), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('K5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('J5')
            elif [(i, i+1), (i, i+3), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('I5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('H5')    
            
        elif len(edges_tb_used) == 5:
            if [(i, i+1), (i, i+2), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('G5')
            elif [(i, i+1), (i, i+4), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('F5')
            elif [(i, i+1), (i+1, i+2), (i+2, i+3), (i+2, i+4), (i+3, i+4)] == edges_tb_used:
                pattern.append('E5')
            elif [(i, i+1), (i, i+3), (i+1, i+2), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('D5')
            elif [(i, i+1), (i+1, i+2), (i+1, i+4), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('C5')
            elif [(i, i+1), (i+1, i+2), (i+1, i+3), (i+2, i+3), (i+3, i+4)] == edges_tb_used:
                pattern.append('B5')
            
        elif len(edges_tb_used) == 4:
            pattern.append('A5')
        # print(pattern)    
        i+=1
    return pattern

In [ ]:
shop_preprocessed_df['pattern_hvg_4_nodes'] = shop_preprocessed_df['HVGms_edges'].apply(pattern_hvg_4)
shop_preprocessed_df['pattern_hvg_5_nodes'] = shop_preprocessed_df['HVGms_edges'].apply(pattern_hvg_5)
shop_preprocessed_df.head()

In [ ]:
shop_preprocessed_df['unigram'] = shop_preprocessed_df['product_action'].apply(lambda x: list(ngrams(x, 1)))
shop_preprocessed_df['bigram'] = shop_preprocessed_df['product_action'].apply(lambda x: list(ngrams(x, 2)))
shop_preprocessed_df['trigram'] = shop_preprocessed_df['product_action'].apply(lambda x: list(ngrams(x, 3)))
shop_preprocessed_df.head()

In [ ]:
print(shop_preprocessed_df.unigram)
print(shop_preprocessed_df.bigram)
print(shop_preprocessed_df.trigram)

In [ ]:
shop_preprocessed_df.to_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/data/shopper_data_feature_extracted.csv', index=False)